In [ ]:
from __future__ import print_function
import os
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

from custom_model.layers_keras import *
from custom_model.model_keras import *
from custom_model.math_utils import *
from utils_vis import *
from keras import metrics

import matplotlib.pyplot as plt
%matplotlib inline
import h5py
import numpy as np

In [ ]:
dt = 5
horizon = 30
OBS = 60//dt-1
PRED = horizon//dt

model_name = 'dfn'

Data = h5py.File('Datasets/RRot_cc5_30.h5', 'r')
x_train = np.array(Data['Speed_obs_train'])
y_train = np.array(Data['Speed_pred_train'])
e_train = np.array(Data['E_train'])
x_test = np.array(Data['Speed_obs_test'])
y_test = np.array(Data['Speed_pred_test'])
e_test = np.array(Data['E_test'])

x_train = x_train[:,-OBS:]
y_train = y_train[:,:PRED]
e_train =e_train[:,:PRED]
y_test = y_test[:,:PRED]
e_test =e_test[:,:PRED]

X_train = z_score(x_train, np.mean(x_train), np.std(x_train))
X_test = z_score(x_test, np.mean(x_test), np.std(x_test))
Y_train = z_score(y_train, np.mean(y_train), np.std(y_train))
Y_test = z_score(x_test, np.mean(y_test), np.std(y_test))
E_train = z_score(e_train, np.mean(e_train), np.std(e_train))
E_test = z_score(e_test, np.mean(e_test), np.std(e_test))

In [ ]:
#create model
encoder = RNN(DGCRNNCell(k=9,dgc_mode='hybrid'), return_state=True)
decoder = RNN(DGCRNNCell(k=9,dgc_mode='hybrid'), return_sequences=True, return_state=True)

unstack_k = Lambda(unstack)
choice = Scheduled()
    
input_obs = Input(shape=(OBS, 208, 1)) 
input_gt = Input(shape=(PRED, 208, 1)) #(None, 4, 208, 1)
encoder_inputs = Lambda(lambda x: K.squeeze(x, axis = -1))(input_obs) # (None, 29, 208)
    
encoder_outputs, state_h = encoder(encoder_inputs)
unstacked = unstack_k(input_gt) #[(None, 208, 1) x 4] list
    
initial = unstacked[0] #(None, 208, 1)
decoder_inputs = Lambda(lambda x: K.permute_dimensions(x, (0,2,1)))(initial) #(None, 1, 208)
decoder_outputs_new, state_h_new = decoder(decoder_inputs, initial_state=state_h)
state_h = state_h_new
feature = []
feature.append(decoder_outputs_new)
if PRED > 1:       
    for i in range(1,PRED):
        decoder_inputs = unstacked[i]#(None, 208, 1)
        decoder_inputs = Lambda(lambda x: K.permute_dimensions(x, (0,2,1)))(decoder_inputs)#(None, 1, 208)
        decoder_outputs_new, state_h_new = decoder(decoder_inputs, initial_state=state_h)
        state_h = state_h_new
        feature.append(decoder_outputs_new)
    
outputs = Lambda(stack)(feature)
model = Model([input_obs, input_gt], outputs)

In [ ]:
#load pretrained models
filename = 'pretrained/cc2-k4.h5'
file=h5py.File(filename,'r')
weight = []
for i in range(len(file.keys())):
    weight.append(file['weight'+str(i)][:])
model.set_weights(weight)

In [ ]:
#prediction
y = model.predict([X_test,E_test])
y =  z_inverse(y, np.mean(y_test), np.std(y_test))

In [ ]:
# extract dynamic kernels from outputs
W = y[...,1:]
# choose each step or all steps in the decoder
step = 0
W = W[:,[step]]
W.shape

In [ ]:
f_avr,J_avr,C = Speed_k_correlation(ground_truth=120*y_train[:,[step]], weights=W, k=4)

dynamicf = Compare(ground_truth=120*y_train[:,[step]].squeeze(), weights=W.squeeze(), k=4)

In [ ]:
plt.figure(figsize = (13,5))
labels = [i for i in range(0, 125, 5)]
#y_lab = [-4.5, -3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5, 4.5]
y_lab = np.arange(-9.5,10.5,1)*200
plt.xlabel("v(km/h)",fontsize=22)
plt.xlim(0,120)
plt.ylim(-9.5*200,9.5*200)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.xticks(np.arange(0, 125, 10))
plt.yticks(np.arange(-1800, 1801, 400))
plt.ylabel("Directional distance (m)",fontsize=22)
plt.pcolormesh(labels, y_lab, np.flip(J_avr,axis=1), cmap = 'jet')
plt.title('Average spatial attention vs speed on RotCC5', fontsize=20)
plt.colorbar()
plt.clim(0,0.2)
#plt.show()
plt.savefig('Figs/hm-cc5.PNG',dpi=600)

In [ ]:
head = 260
end = head+150
s = 208
gt = 120*y_train[:,[step]].squeeze()

plt.figure(figsize = (15,8))
y_lab = [0.2*i for i in range(s)]
x_lab = [14+3/60*i for i in range(150)]
#y, x = np.mgrid[slice(0, (s+1)*0.2, 0.2), slice(14-4/60., 14+150*2/60., 2/60)]
plt.xlabel("time (PM)",fontsize=22)
plt.xlim(14,19)
plt.ylim(0,(s-0.2)*0.2)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.ylabel("position (km)",fontsize=22)
#plt.pcolormesh(x_lab, y_lab, 120*y_train[:,1][head:end].transpose(), cmap = 'autumn')
plt.pcolormesh(x_lab, y_lab, dynamicf[head:end].transpose()/4, cmap = 'coolwarm')
#levels1 = MaxNLocator(nbins=120).tick_values(-3, 3)
#plt.contourf(x[:-1, :-1] + 0.2/2.,
                  #y[:-1, :-1] + 2/120., [head:end].transpose(), levels=levels1,
                  #cmap='RdYlBu_r')
#plt.title('Speed evolution on 13-11-2018 (RotCC2)', fontsize=20)
plt.title('Attention coefficient evolution on 13-11-2018 (RotCC2)', fontsize=20)
plt.colorbar()
#plt.clim(0,1)
#plt.show()
plt.savefig('Figs/attention.PNG',dpi=600)